The following document has information about my process of experimenting with various models to solve a classification problem, in which I tested different models of supervised and unsupervised learning. Which then save in a file to be consumed after training.

As it is a proof of concept the code is not ordered and neither optimized.

In [ ]:
import pandas as pd
import os
from os import walk
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier

import xgboost as xgb
from xgboost import plot_tree

import joblib

import tensorflow as tf

import keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras import Sequential
from keras.layers import Dense
from keras.models import load_model

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'credentials.json'

In [ ]:
query_url = """

WITH urls_main_table AS (
    SELECT 
    source,   
    ad_id,   
    image_asset_url as url
    FROM `omg-latam-prd-adh-datahouse-cl.clients_table.main_url_dh`
    where source = 'Facebook ads' and image_asset_url != '0'
    group by 1,2,3  

    UNION ALL 

    SELECT 
    source,   
    ad_id,   
    promoted_tweet_card_image_url as url
    FROM `omg-latam-prd-adh-datahouse-cl.clients_table.main_url_dh`
    where source = 'Twitter ads' and promoted_tweet_card_image_url != '0'
    group by 1,2,3  

),

ad_values AS (
    SELECT date,
    ad_id, spend, post_engagements as clicks
    FROM `main_views_tables.main_ad`
    where regexp_contains(account_name, '(?i).*scoti.*')
),

spend_average AS (
    select ad_id, avg(spend) as avg_spend, avg(post_engagements) as avg_clicks from `main_views_tables.main_ad`
    group by 1
),

categorical_feats AS (

    SELECT ad_id, main_topic, main_color, locale, second_topic, second_color, third_topic, third_color
    FROM `clients_table.main_categorical_values`
)



SELECT 

date, source, url, e.ad_id,
main_topic, main_color, locale, second_topic, second_color, third_topic, third_color,

(CASE 
WHEN spend >= avg_spend THEN 1
WHEN spend < avg_spend THEN 0
END) as over_avg_spend, 

(CASE 
WHEN clicks >= avg_clicks THEN 1
WHEN clicks < avg_clicks THEN 0
END) as over_avg_clicks

FROM
(SELECT
date, source, url, c.ad_id,
main_topic, main_color, locale, second_topic, second_color, third_topic, third_color, spend, clicks
FROM (SELECT date,
a.source, a.url, b.ad_id, b.spend, b.clicks
FROM urls_main_table a
RIGHT JOIN ad_values b
ON a.ad_id = b.ad_id) c

INNER JOIN categorical_feats d

ON c.ad_id = d.ad_id) e

INNER JOIN spend_average f
ON e.ad_id = f.ad_id

"""

dataframe = pd.read_gbq(query_url)

In [ ]:
dataframe = dataframe[['main_topic', 'main_color', 'second_topic', 'second_color', 'third_topic', 'third_color', 'locale', 'over_avg_spend', 'over_avg_clicks']]

In [ ]:
vars = ['main_topic', 'main_color', 'second_topic', 'second_color', 'third_topic', 'third_color', 'locale']

for var in vars:
  cat_list = 'var' + '_' + var
  cat_list = pd.get_dummies(dataframe[var], prefix= var)
  dataframe1 = dataframe.join(cat_list)
  dataframe = dataframe1

dataframe

,main_topic,main_color,second_topic,second_color,third_topic,third_color,locale,over_avg_spend,over_avg_clicks,main_topic_Arm,main_topic_Chin,main_topic_Clothing,main_topic_Eyelash,main_topic_Forehead,main_topic_Glasses,main_topic_Hair,main_topic_Hand,main_topic_Head,main_topic_Jaw,main_topic_Laptop,main_topic_Mobilephone,main_topic_Plant,main_topic_Shorts,main_topic_Sleeve,main_topic_Smile,main_topic_Table,main_topic_Tire,main_topic_video,main_color_black,main_color_cadetblue,main_color_darkolivegreen,main_color_darkslategray,main_color_dimgray,main_color_dodgerblue,main_color_hotpink,main_color_linen,main_color_mediumpurple,main_color_sienna,main_color_skyblue,main_color_yellowgreen,...,second_color_slategray,third_topic_Beard,third_topic_Cap,third_topic_Chin,third_topic_Eyelash,third_topic_Eyewear,third_topic_Flashphotography,third_topic_Font,third_topic_Furniture,third_topic_Gesture,third_topic_Jaw,third_topic_Jersey,third_topic_Landvehicle,third_topic_Organism,third_topic_Outerwear,third_topic_Personalcomputer,third_topic_Skin,third_topic_Sleeve,third_topic_Smile,third_topic_Sportsuniform,third_topic_Tabletcomputer,third_topic_Tableware,third_color_black,third_color_crimson,third_color_darkgray,third_color_darkslategray,third_color_dimgray,third_color_gray,third_color_lightgray,third_color_maroon,third_color_mistyrose,third_color_silver,third_color_slateblue,third_color_slategray,third_color_tan,locale_en,locale_error,locale_es,locale_fr,locale_no_text
0,Arm,dodgerblue,Shirt,skyblue,Smile,lightgray,en,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,Smile,mediumpurple,Product,black,Sleeve,slateblue,en,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
2,Table,hotpink,Fashion,darkgray,Tableware,mistyrose,en,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,video,None,None,None,None,None,error,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Laptop,cadetblue,Product,seagreen,Cap,gray,en,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16489,Chin,black,Jaw,darkslategray,Beard,darkslategray,es,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
16490,Chin,black,Jaw,darkslategray,Beard,darkslategray,es,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
16491,Chin,black,Jaw,darkslategray,Beard,darkslategray,es,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
16492,Chin,black,Jaw,darkslategray,Beard,darkslategray,es,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
data_vars = dataframe.columns.values.tolist()
to_keep = [i for i in data_vars if i not in vars]

data_final = dataframe[to_keep]
data_final.columns.values

data_final_vars = data_final.columns.values.tolist()
v_y = ['over_avg_clicks']
v_x = [i for i in data_final_vars if i not in v_y]

In [ ]:
X = dataframe[v_x]
y = dataframe[v_y]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 40, stratify = y)

In [ ]:
model_xgb = xgb.XGBClassifier(objective = 'binary:logistic', n_estimators = 105, max_depth=75, seed = 10, reg_alpha=7)
model_xgb.fit(X_train.values, y_train.values.ravel())
y_pred = model_xgb.predict(X_test.values)

In [ ]:
print('Area debajo de la curva:', roc_auc_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('accuracy:', round(accuracy_score(y_test, y_pred), 2))

print(confusion_matrix(y_test, y_pred))

Area debajo de la curva: 0.9986280487804878
Recall: 0.9972560975609757
Precision: 1.0
accuracy: 1.0
[[  38    0]
 [  18 6542]]


In [ ]:
base_frame = dict.fromkeys(list(X_train.columns), [0])

main_topic = 'Clothing' #@param ['Table', 'Smile','Laptop', 'video', 'Arm', 'Hair', 'Jaw', 'Forehead','Head','Chin','Clothing','Sleeve','Plant','Tire', 'Eyelash','Hand', 'Mobilephone', 'Glasses','Shorts']
second_topic = 'Jaw' #@param ['Jaw', 'Product', 'Flashphotography', 'Sleeve', 'Beard', 'Computer', 'Glasses', 'Visioncare', 'Fashion', 'Shirt', 'Jeans', 'Wheel', 'Jersey', 'Smile', 'CommunicationDevice', 'Plant', 'Mobilephone', 'Green', 'Chin', 'Human']
third_topic = 'Font' #@param ['Beard', 'Font', 'Jersey', 'Jaw', 'Chin', 'Eyewear', 'Sleeve', 'Cap', 'Smile', 'Tableware', 'Personalcomputer', 'Eyelash', 'Skin',  'Landvehicle', 'Tabletcomputer', 'Gesture', 'Organism', 'Outerwear', 'Flashphotography', 'Sportsuniform', 'Furniture']

locale = 'en' #@param ['es', 'en']

main_color = 'skyblue' #@param ['black', 'darkslategray', 'darkolivegreen', 'cadetblue', 'dodgerblue', 'mediumpurple', 'hotpink', 'skyblue', 'dimgray', 'linen', 'yellowgreen', 'sienna']
second_color = 'darkgray' #@param ['darkslategray', 'dimgray', 'black', 'darkgray', 'seagreen','skyblue', 'maroon', 'paleturquoise', 'silver', 'crimson','darkgreen', 'slategray', 'mediumpurple', 'gray']
third_color = 'lightgray' #@param ['darkslategray', 'slateblue', 'lightgray', 'mistyrose', 'gray', 'maroon', 'black', 'tan', 'darkgray', 'crimson', 'slategray', 'dimgray', 'silver']

over_avg_spend = 'False' #@param ['True', 'False']
if over_avg_spend == 'False':
  over_avg_spend = 0
else:
  over_avg_spend = 1

dataframe = pd.DataFrame({'main_topic':[main_topic], 'main_color':[main_color], 'second_topic':[second_topic], 'second_color':[second_color], 
                          'third_topic':[third_topic], 'third_color':[third_color], 'locale':[locale], 'over_avg_spend':[over_avg_spend]})

vars = ['main_topic', 'main_color', 'second_topic', 'second_color', 'third_topic', 'third_color', 'locale']

for var in vars:
  cat_list = 'var' + '_' + var
  cat_list = pd.get_dummies(dataframe[var], prefix= var)
  dataframe1 = dataframe.join(cat_list)
  dataframe = dataframe1

data_vars = dataframe.columns.values.tolist()
to_keep = [i for i in data_vars if i not in vars]

data_final = dataframe[to_keep]
my_dict_frame = data_final.to_dict('records')

base_frame.update(my_dict_frame[0])
to_predict_frame = pd.DataFrame(base_frame)

result = model_xgb.predict(to_predict_frame.values)
result_prob = model_xgb.predict_proba(to_predict_frame.values)

print('If the ad can be above the average of the results studied it will be classified as 1, otherwise it will be classified as 0. \nThe classification of the ad model entered is: {}'.format(result))

print('Probability of not being successful according to the given parameters: {}%'.format(result_prob[0][0] * 100),
      '\nProbability of success according to the given parameters: {}%'.format(result_prob[0][1]*100))

If the ad can be above the average of the results studied it will be classified as 1, otherwise it will be classified as 0. 
The classification of the ad model entered is: [1]
Probability of not being successful according to the given parameters: 0.09180307388305664% 
Probability of success according to the given parameters: 99.90819692611694%


In [ ]:
plot_tree(model_xgb, num_trees=75)
plt.show()

### FIRST MDOEL

In [ ]:
model_classifier = Sequential()

input_dimension = len(v_x)
layers = 6
neurons = 16
output_layer = 1

for layer in range(layers):
  model_classifier.add(Dense(neurons, activation= 'relu', kernel_initializer= 'random_normal'))
  
  if layer == layers-1:
    model_classifier.add(Dense(output_layer, activation= 'sigmoid', kernel_initializer='random_normal'))

define_optimizer = keras.optimizers.Adam(lr = 0.01)
model_classifier.compile(optimizer = define_optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

history_values = model_classifier.fit(x = X_train.values, y = y_train.values, batch_size = 15, epochs = 350, validation_split = 0.15)

Epoch 1/350
557/557 [==============================] - 2s 3ms/step - loss: 0.1190 - accuracy: 0.9796 - val_loss: 0.0056 - val_accuracy: 0.9959
Epoch 2/350
557/557 [==============================] - 1s 2ms/step - loss: 0.0071 - accuracy: 0.9937 - val_loss: 0.0047 - val_accuracy: 0.9973
Epoch 3/350
557/557 [==============================] - 1s 2ms/step - loss: 0.0056 - accuracy: 0.9976 - val_loss: 0.0046 - val_accuracy: 0.9973
Epoch 4/350
557/557 [==============================] - 1s 2ms/step - loss: 0.0049 - accuracy: 0.9985 - val_loss: 0.0046 - val_accuracy: 0.9973
Epoch 5/350
557/557 [==============================] - 1s 2ms/step - loss: 0.0053 - accuracy: 0.9982 - val_loss: 0.0046 - val_accuracy: 0.9973
Epoch 6/350
557/557 [==============================] - 1s 2ms/step - loss: 0.0054 - accuracy: 0.9974 - val_loss: 0.0047 - val_accuracy: 0.9973
Epoch 7/350
557/557 [==============================] - 1s 2ms/step - loss: 0.0046 - accuracy: 0.9979 - val_loss: 0.0048 - val_accuracy: 0.9973

In [ ]:
model_classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                1696      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 1

In [ ]:
evaluation_model = model_classifier.evaluate(X_train, y_train)
print(evaluation_model)

308/308 [==============================] - 0s 1ms/step - loss: 0.0049 - accuracy: 0.9977
[0.004854610655456781, 0.9976599812507629]


In [ ]:
y_pred = model_classifier.predict(X_test).ravel()
y_pred = (y_pred > 0.5)

In [ ]:
print('Area debajo de la curva:', roc_auc_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('accuracy:', round(accuracy_score(y_test, y_pred), 2))

Area debajo de la curva: 0.9980813507290867
Recall: 0.9961627014581734
Precision: 1.0
accuracy: 1.0


In [ ]:
print(confusion_matrix(y_test, y_pred))

[[  38    0]
 [  25 6490]]


## KERAS SKLEARN

In [ ]:
def create_model(init='random_normal'):

  model = Sequential()
  model.add(Dense(16, kernel_initializer=init, activation='relu'))
  model.add(Dense(16, kernel_initializer=init, activation='relu'))
  model.add(Dense(16, kernel_initializer=init, activation='relu'))
  model.add(Dense(16, kernel_initializer=init, activation='relu'))
  model.add(Dense(16, kernel_initializer=init, activation='relu'))
  model.add(Dense(16, kernel_initializer=init, activation='relu'))
  model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
  define_optimizer = keras.optimizers.Adam(lr = 0.01)
  model.compile(loss='binary_crossentropy', optimizer=define_optimizer, metrics=['accuracy'])
  return model
 

model = KerasClassifier(build_fn=create_model, verbose=0, validation_split=0.35)
model._estimator_type = "classifier"
init = ['random_normal']
epochs = [350]
batches = [15]

param_grid = dict(epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

print(means, stds, params)

Best: 0.997660 using {'batch_size': 15, 'epochs': 350, 'init': 'random_normal'}
[0.99765997] [0.00051882] [{'batch_size': 15, 'epochs': 350, 'init': 'random_normal'}]


In [ ]:
y_pred = grid_result.predict(X_test)

print('Area debajo de la curva:', roc_auc_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('accuracy:', round(accuracy_score(y_test, y_pred), 2))
print('')
print(confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Area debajo de la curva: 0.9980813507290867
Recall: 0.9961627014581734
Precision: 1.0
accuracy: 1.0

[[  38    0]
 [  25 6490]]


### OTHER MODEL

In [ ]:
tree_model = RandomForestClassifier()

param_grid = {'criterion':['gini', 'entropy'], 'max_depth': np.arange(5,15,5), 'min_samples_leaf': np.arange(1,15,10), 'n_estimators': np.arange(5,105,10), 'ccp_alpha':[0.0, 0.1,0.5,0.9, 1]}

grid_tree_model = GridSearchCV(tree_model, param_grid, cv = 10)
grid_tree_model.fit(X_train.values, y_train.values.ravel())

print('Tuned Decision Tree: {}'.format(grid_tree_model.best_params_))
print('Tuned accuracy: {}'.format(grid_tree_model.best_score_))

Tuned Decision Tree: {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 5}
Tuned accuracy: 0.9982702894211783


In [ ]:
y_pred = grid_tree_model.predict(X_test.values)

In [ ]:
print('Area debajo de la curva:', roc_auc_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('accuracy:', round(accuracy_score(y_test, y_pred), 2))

Area debajo de la curva: 0.9726865129054408
Recall: 0.9980046047582501
Precision: 0.9996924969249692
accuracy: 1.0


In [ ]:
print(confusion_matrix(y_test, y_pred))

[[  36    2]
 [  13 6502]]


### OTHER XGBOOST

In [ ]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(objective = 'binary:logistic', n_estimators = 105, max_depth=75, seed = 10, reg_alpha=7)
model_xgb.fit(X_train.values, y_train.values.ravel())
y_pred = model_xgb.predict(X_test.values)

In [ ]:
print('Area debajo de la curva:', roc_auc_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))

Area debajo de la curva: 0.9980813507290867
Recall: 0.9961627014581734
Precision: 1.0


In [ ]:
round(accuracy_score(y_test, y_pred), 2)

1.0

In [ ]:
print(confusion_matrix(y_test, y_pred))

[[  38    0]
 [  25 6490]]


### SKLEARN NN

In [ ]:
nn_class = MLPClassifier()

param_grid = {'hidden_layer_sizes':[(8,8,8,8,8,8,8), (16,16,16,16,16,16)], 
              'activation': ['relu', 'logistic', 'tanh'],
              'solver': ['lbfgs', 'adam'],
              'max_iter': [800],
              'learning_rate': ['constant', 'adaptive']
              }

grid_nn_class = GridSearchCV(nn_class, param_grid, cv = 10)
grid_nn_class.fit(X_train.values, y_train.values.ravel())

GridSearchCV(cv=10, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_stat...
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'activation': ['relu', 'logistic', 'tanh'],
                         'hidden_lay

In [ ]:
print('Tuned Decision Tree: {}'.format(grid_nn_class.best_params_))
print('Tuned accuracy: {}'.format(grid_nn_class.best_score_))

Tuned Decision Tree: {'activation': 'tanh', 'hidden_layer_sizes': (8, 8, 8, 8, 8, 8, 8), 'learning_rate': 'constant', 'max_iter': 800, 'solver': 'adam'}
Tuned accuracy: 0.9982702894211783


In [ ]:
y_pred = grid_nn_class.predict(X_test.values)

In [ ]:
print('Area debajo de la curva:', roc_auc_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('accuracy:', round(accuracy_score(y_test, y_pred), 2))

Area debajo de la curva: 0.9726865129054408
Recall: 0.9980046047582501
Precision: 0.9996924969249692
accuracy: 1.0


In [ ]:
print(confusion_matrix(y_test, y_pred))

[[  36    2]
 [  13 6502]]


### VOTING

In [ ]:
client = 'someclient'

model_classifier.save("model.h5") # keras

joblib.dump(model_xgb, 'xgboost_tree' + client +'.pkl') #xgboost
joblib.dump(grid_tree_model, 'sklearn_tree' + client +'.pkl') #sklearn
joblib.dump(grid_nn_class, 'sklearn_nn'+ client +'.pkl') #sklearn

['sklearn_nnfedex.pkl']

In [ ]:
_, _, filenames = next(walk('/content'))